<a href="https://colab.research.google.com/github/Hamza-Ali0237/HF-Projects/blob/main/Beginner/Text%20Summarization%20App/Text_Summarization_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline

In [ ]:
summarizer = pipeline(
    task = "summarization",
    model = "facebook/bart-large-cnn",
    device = 0
)

In [ ]:
# Sample text
text = """In the heart of a sprawling metropolis, where steel and glass pierce the sky,
          a vibrant energy pulses day and night. Yellow taxis weave through a labyrinth of bustling streets,
          their horns adding to the symphony of city life. Street vendors hawk their wares,
          their voices rising above the chatter of hurried footsteps. Amidst the towering skyscrapers,
          pockets of green offer a respite, where weary souls seek a moment of tranquility. The city never sleeps,
          a melting pot of dreams and ambitions, constantly evolving and reaching for the future."""

In [ ]:
summary = summarizer(text, max_length = 50)

In [ ]:
print(summary)

In [ ]:
# Extract exact answer
answer = summary[0]['summary_text']
print(answer)

In [ ]:
# check on user input
user_text = input("Enter text to summarize: ")
custom_max_length = int(input("Enter max length: "))
custom_summary = summarizer(user_text, max_length = custom_max_length)

In [ ]:
print(custom_summary[0]['summary_text'])

In [ ]:
# Saving model for later use

# Extract model
model = summarizer.model
# Extract tokenizer
tokenizer = summarizer.tokenizer

# Save both to a directory
save_directory = "saved_summarizer_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Gradio UI

In [ ]:
!pip install gradio

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import gradio as gr

In [ ]:
# Load the saved model and tokenizer
model_path = "saved_summarizer_model"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# recreate pipeline
summarizer = pipeline(
    task = "summarization",
    model = model,
    tokenizer = tokenizer,
    device = 0
)

In [ ]:
# Define the Gradio interface function
def summarize_text(text, max_length=100):
  summary = summarizer(text, max_length = max_length, min_length=25, do_sample = False)
  return summary[0]['summary_text']

# Gradio UI
demo = gr.Interface(
    fn = summarize_text,
    inputs = [
        gr.Textbox(label="Enter Text to Summarize", lines=10, placeholder="Paste a long paragraph here..."),
        gr.Slider(30, 200, value=100, step=10, label="Max Summary Length")
    ],
    outputs = gr.Textbox(label="Summary"),
    title = "Text Summarization",
    description = "Enter a long paragraph and get a concise summary."
)

demo.launch()